In [3]:
import numpy as np
import matplotlib as plt

import IPythonMagic
from Timer import Timer

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

In [4]:
%setup_logging

Global logger already initialized!


In [5]:
%cuda_context_handler context

Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10699 / 11441 MB available
Created context handle <52426688>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Stefano_B/MilanoGPU2018/notebooks/cuda_cache


In [11]:
kernel_src = """

__global__ void shmemReduction(float* output, float* input, int size) {
    // 1) stride through global memory and compute the maximum for every thread
    int gid = blockIdx.x*blockDim.x + threadIdx.x; // note: blockIdx.x is always zero, because we use only one block
    
    // float max = 0; // IS IT THE MINIMUM? BETTER THIS:
    // float max_value = -FLT_MAX;
    float max_value = -9999999;
    for (int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    
    // tmp write to memory to check if things work so far
    output[threadIdx.x] = max_value;
    
    // 2) store the per-thread maximum in shared memory
    __shared__ float max_shared[32];
    max_shared[threadIdx.x] = max_value;
    
    // 3) synchronize so that all threads see the same shared memory
    __syncthreads();
    
    // 4) find maximum in shared memory
    
    // Reduce from 32 to 16 elements
    if(threadIdx.x < 16) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x+16]);
    }
    
    // Reduce from 16 to 8 elements
    if(threadIdx.x < 8) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x+8]);
    }
        
    // Reduce from 8 to 4 elements
    if(threadIdx.x < 4) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x+4]);
    }    
    
    // Reduce from 4 to 2 elements
    if(threadIdx.x < 2) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x+2]);
    }    
    
    // Reduce from 2 to 1 elements
    if(threadIdx.x < 1) {
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x], max_shared[threadIdx.x+1]);
    }    
    
    // 5) write to output
    if(threadIdx.x == 0) {
      output[0] = max_shared[0];
    }
}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

In [12]:
n = 64
a = np.random.random((1, n)).astype(np.float32)

a_g = GPUArray(a.shape, a.dtype)
a_g.set(a)

num_threads = 32
b = np.empty((1,num_threads), dtype=np.float32)

b_g = GPUArray(b.shape, b.dtype)

In [15]:
block_size = (num_threads, 1, 1)
grid_size  = (1, 1, 1)

kernel_function(b_g, a_g, np.int32(n), grid=grid_size, block=block_size)
b_g.get(b)

print(a)
print(b)
print(np.max(a))

[[0.83689755 0.41948807 0.87312925 0.29224232 0.9481355  0.4531257
  0.62819946 0.61715794 0.7371609  0.7659136  0.28334272 0.39048427
  0.8993981  0.44780144 0.79649544 0.5367203  0.72738224 0.25699103
  0.14175162 0.54631895 0.24623673 0.728681   0.5996733  0.23144549
  0.20511219 0.5677549  0.9835198  0.10674427 0.5630324  0.2249372
  0.01246603 0.7465469  0.71590334 0.6645421  0.22940327 0.08933225
  0.7106722  0.1794032  0.26399305 0.66364807 0.51912904 0.43102342
  0.86559397 0.29227048 0.23926395 0.10237733 0.381412   0.11328582
  0.6486532  0.6512805  0.70482665 0.98614544 0.5299481  0.03833594
  0.12147699 0.25330898 0.4995537  0.44192547 0.32496405 0.75397974
  0.65167975 0.08425208 0.09387032 0.89053136]]
[[0.98614544 0.6645421  0.87312925 0.29224232 0.9481355  0.4531257
  0.62819946 0.66364807 0.7371609  0.7659136  0.86559397 0.39048427
  0.8993981  0.44780144 0.79649544 0.5367203  0.72738224 0.6512805
  0.70482665 0.98614544 0.5299481  0.728681   0.5996733  0.25330898
  0.